reviewing pytorch fundamentals version 2

In [2]:
import torch
print(torch.__version__)

2.9.1+cpu


In [3]:
tensor0d=torch.tensor(1) # scaler
tensor1d=torch.tensor([1,2,3]) # vector
tensor2d=torch.tensor([[1,2],[3,4]]) # tensor 2D
tensor3d=torch.tensor([[[1,2],[3,4]],[[5,6],[7,8]]]) # tensor 3D


In [4]:
print(tensor0d) 
print(tensor1d)
print(tensor2d)
print(tensor3d)

tensor(1)
tensor([1, 2, 3])
tensor([[1, 2],
        [3, 4]])
tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])


In [5]:
# data type of tensor
tensor1d = torch.tensor([1,2,3]) 
print(tensor1d.dtype)


torch.int64


In [6]:
float_tensor=torch.tensor([1.0,2.0,3.0])
print(float_tensor.dtype)

torch.float32


In [7]:
# change form 1 data type into another is also possible
float_vec=tensor1d.to(torch.float32)
print(float_vec)
print(float_vec.dtype)

tensor([1., 2., 3.])
torch.float32


common pytorch operations

In [8]:
tensor2d = torch.tensor([[1,2,3],[4,5,6]])
print(tensor2d.shape) 


torch.Size([2, 3])


In [9]:
print(tensor2d.reshape([3,2]))
# reshape used to change the shape of tensor

tensor([[1, 2],
        [3, 4],
        [5, 6]])


In [10]:
# most common while working with deep learning is the use of .view()
print(tensor2d.view(3,2))




tensor([[1, 2],
        [3, 4],
        [5, 6]])


In [11]:
# aslo we can do Transpose (T) for same
print(tensor2d.T)

tensor([[1, 4],
        [2, 5],
        [3, 6]])


In [12]:
# for matrix multiplication
print(tensor2d.matmul(tensor2d.T))
print(tensor2d@tensor2d.T)


tensor([[14, 32],
        [32, 77]])
tensor([[14, 32],
        [32, 77]])


seeing models as computational graphs

In [13]:
import torch.nn.functional as F

y=torch.tensor([1.0])
x1=torch.tensor([1.1])
w1=torch.tensor([2.2])
b=torch.tensor([0.0])
z=x1 * w1 + b
a=torch.sigmoid(z)

loss=F.binary_cross_entropy(a,y)
print(loss)


tensor(0.0852)


In [14]:
# for taking the gradients , we need to ensure giving required grad = True
# by doing this pytorch's automatically internally calculates
# the respective gradients for training a neural network

import torch.nn.functional as F
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)
z = x1 * w1 + b
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a, y)

grad_l_w1=grad(loss, w1, retain_graph=True)
grad_l_b=grad(loss,b,retain_graph=True)



In [15]:
print(grad_l_w1)
print(grad_l_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [16]:
# for back propagation
loss.backward()

In [17]:
print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


Implementing a multilayer neural network

In [18]:
# all the custom neural network architecture in pytorch 
# can be made with 'torch.nn.Module' class with '__init__' constructor

class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers=torch.nn.Sequential(

            # 1st hidden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(30,20),
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(20,num_outputs),

        )

    def forward(self, x):
        logits=self.layers(x)
        return logits
    



In [19]:
torch.manual_seed(123)
model=NeuralNetwork(50,3)

print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [20]:
# to check the total number of trainable parameters

num_params=sum(p.numel() for p in model.parameters() if p.requires_grad)
print(num_params)


2213


In [21]:
# we can also see the parametes inside the models
print(model.layers[0].weight)
print(model.layers[4].weight)


Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)
Parameter containing:
tensor([[-0.1692, -0.0858,  0.2063, -0.1134, -0.1537,  0.0388,  0.1798, -0.0861,
         -0.0695, -0.0770, -0.1165,  0.0031, -0.0258,  0.0932, -0.1665, -0.0642,
         -0.2058, -0.2228,  0.0619, -0.0289],
        [-0.2011, -0.0067,  0.0681,  0.1624,  0.1344, -0.2128,  0.0601, -0.2104,
          0.1657,  0.1458, -0.2062,  0.0016,  0.0697,  0.0092, -0.2019,  0.2094,
         -0.1101,  0.0695,  0.1398, -0.2045],
        [-0.0663, -0.1034,  0.2191, -0.0906, -0.1899, -0.2023, -0.0230,  0.1957

In [22]:
# seeing the parameters
print(model.layers[0].weight.shape)
print(model.layers[0].bias)
print(model.layers[0].bias.shape)


torch.Size([30, 50])
Parameter containing:
tensor([-0.1250,  0.0513,  0.0366,  0.0075,  0.0509,  0.0545, -0.0393,  0.0924,
        -0.1412, -0.1232, -0.1063,  0.0081, -0.1249,  0.0101, -0.0019, -0.1298,
         0.1388, -0.0330,  0.1017,  0.1247, -0.0554, -0.0417,  0.1388,  0.0159,
         0.1215,  0.0385,  0.0769, -0.1224, -0.0279,  0.0991],
       requires_grad=True)
torch.Size([30])


In [51]:
# this way we can pass a value in forward network
torch.manual_seed(123)
X=torch.rand((1,50))
out=model(X)
print(out)


tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)


In [24]:
# for not using any backpropagation
# when we want to test it

with torch.no_grad():
    out=model(X)
print(out)


tensor([[-0.1262,  0.1080, -0.1792]])


In [25]:
# for fiding out the class membership probablities
# we'll use softmax

with torch.no_grad():
    output=torch.softmax(model(X),dim=1)
print(output)

# all are sum up to 1

tensor([[0.3113, 0.3934, 0.2952]])


setting up efficient data loaders

In [28]:
# each dataset has to be converted into data loaders
# in dataloaders, the shuffing, assembling batches and more happened for training a neural network model


X_train=torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])

y_train=torch.tensor([0,0,0,1,1])

X_test=torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6]
])

y_test=torch.tensor([0, 1])




In [29]:
# defining custom dataset classes
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self,X,y):
        self.features=X
        self.labels=y

    def __getitem__(self, index):
        one_x=self.features[index]
        one_y=self.labels[index]
        return one_x, one_y
    
    def __len__(self):
        return self.labels.shape[0]
        


In [33]:
train_ds=ToyDataset(X_train, y_train)
test_ds=ToyDataset(X_test, y_test)

print(train_ds[2])
print(len(train_ds))

(tensor([-0.5000,  2.6000]), tensor(0))
5


In [49]:
# now from dataset to dataloader
from torch.utils.data import DataLoader
torch.manual_seed(123)

train_loader=DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True # practical usage
)

test_loader=DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)



In [50]:
for idx, (x,y) in enumerate(train_loader):
    print(f"Batch {idx+1}:",x,y)
    


Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
